In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-xai

from langchain_xai.chat_models import ChatXAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.


In [ ]:
!pip install timeout-decorator

import json
import numpy as np

from langchain_core.prompts.prompt import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
import timeout_decorator
import time

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

Initialize the model

In [ ]:
import os

# Set the environment variables
os.environ["XAI_API_KEY"] = "..." # Your API key here

# Select a model
#XAI_CHAT_MODEL = "grok-4-0709"
XAI_CHAT_MODEL = "grok-4-fast"

llm = ChatXAI(
    model=XAI_CHAT_MODEL,
    temperature=0.7
)

parser = StrOutputParser()

Open the data

In [ ]:
f = open("drive/MyDrive/data/defi-text-mine-egc-2026/train_v2.jsonl", "r")

text = []
acronyms = []
options = []

for line in f:
  try:
      data = json.loads(line)

      text.append(data["text"])

      acronyms.append(data["acronym"])

      options.append(data["options"])

  except ValueError:
      print('Invalid input:',line)

f.close()

# An LLM based approach

Define the template

In [ ]:
template = '''Find what is "{ACRONYM}" in this phrase: "{TEXT}". Evaluate the following options by giving the boolean list as an answer: '{OPTIONS}'. Output only the boolean list. Do not output any explanation nor other text'''

prompt = PromptTemplate.from_template(template)

Define the pipeline

In [ ]:
chain = prompt | llm | parser

@timeout_decorator.timeout(120, use_signals=False)  # Set the timeout to 120 seconds
def invoke_with_timeout(ACRONYM,TEXT,OPTIONS):
    return chain.invoke({"ACRONYM": ACRONYM, "TEXT": TEXT, "OPTIONS": OPTIONS})

def prediction(ACRONYM,TEXT,OPTIONS):
    try:
        response = invoke_with_timeout(ACRONYM,TEXT,OPTIONS)  # Try to invoke within the timeout
    except timeout_decorator.timeout_decorator.TimeoutError:
        print(f"Timeout occurred for ACRONYM: {ACRONYM}. Retrying...")
        time.sleep(2)  # Optional sleep time before retrying
        try:
            response = invoke_with_timeout(ACRONYM,TEXT,OPTIONS)  # Retry the operation
        except:
            return 'error'

    return response

Function to post-process the results

In [ ]:
import re
import ast

def extract_bool_list(text):
    """Return a list of all boolean lists found in `text`.
       Example: ' [ASSISTANT] [False, True]' -> [[False, True]]"""
    text = text.replace('"','').replace("'","") # get rid of all quotes
    candidates = re.findall(r'\[.*?\]', text)  # non-greedy bracket matches
    results = []
    for cand in candidates:
        # normalize lowercase true/false to Python booleans (optional)
        cand_norm = re.sub(r'\btrue\b', 'True', cand, flags=re.IGNORECASE)
        cand_norm = re.sub(r'\bfalse\b', 'False', cand_norm, flags=re.IGNORECASE)

        try:
            val = ast.literal_eval(cand_norm)
        except Exception:
            # not a Python literal (e.g. [ASSISTANT]) -> skip
            continue

        # keep only lists made entirely of booleans
        if isinstance(val, list) and all(isinstance(x, bool) for x in val):
            results.append(val)

    return results[0] if results else None

Perform prediction (on a train set!)

In [ ]:
result_dic = {}
examples_to_process = [(i,0) for i in range(len(text))]

flag = False
count = 0
while examples_to_process != []:
  if count % 10 == 0:
    print(len(examples_to_process))

  item = examples_to_process.pop(0)

  # Querry the LLM
  result = prediction(acronyms[item[0]],text[item[0]],str([j for j in options[item[0]].keys()]))

  # Post-process result
  labels_pred = extract_bool_list(result)
  if labels_pred != None:
    if len(labels_pred) == len(options[item[0]]):
      result_dic[item[0]] = (labels_pred,item[1])
    else:
      flag = True
  else:
    flag = True

  if flag:
    if item[1] < 1:
      examples_to_process.append((item[0],item[1]+1))
    flag = False
  count+=1

Predicted labels

In [ ]:
result_dic

In [ ]:
len(result_dic)

491

# Evaluation

In [ ]:
from sklearn.metrics import f1_score

f1_list = []

for i in range(len(acronyms)):
  if i in result_dic.keys():
    y_pred = result_dic[i][0]
  else:
    y_pred = np.zeros(len(options[i]),bool)
  y_true = list(options[i].values())
  f1_list.append(f1_score(y_true,y_pred,average='binary'))

print(np.mean(f1_list))

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metr

0.7777777777777777


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predict labels for the test set

Open test set

In [ ]:
f = open("drive/MyDrive/data/defi-text-mine-egc-2026/test_v4.jsonl", "r")

ids = []
text = []
acronyms = []
options = []

for line in f:
  try:
      data = json.loads(line)

      ids.append(data["id"])

      text.append(data["text"])

      acronyms.append(data["acronym"])

      options.append(data["options"])

  except ValueError:
      print('Invalid input:',line)

f.close()

Perform prediction

In [ ]:
val_results = {}
examples_to_process = [(i,0) for i in range(len(text))]

flag = False
count = 0
while examples_to_process != []:
  if count % 10 == 0:
    print(len(examples_to_process))

  item = examples_to_process.pop(0)

  # Querry the LLM
  result = prediction(acronyms[item[0]],text[item[0]],str(options[item[0]]))

  # Process the result
  labels_pred = extract_bool_list(result)

  if labels_pred != None:
    if len(labels_pred) == len(options[item[0]]):
      val_results[item[0]] = (labels_pred,item[1])
    else:
      flag = True
  else:
    flag = True

  if flag:
    if item[1] < 0:
      examples_to_process.append((item[0],item[1]+1))
    flag = False
  count+=1

In [ ]:
val_results

In [ ]:
len(val_results)

Save to the CSV file

In [ ]:
import csv

f_out = open('drive/MyDrive/data/defi-text-mine-egc-2026/submission_grok4_fast.csv', 'w', newline='', encoding='UTF8')

tab = csv.writer(f_out,delimiter=',')

header = ['id','prediction']
tab.writerow(header)

for id in ids:
    if id not in val_results.keys():
      tab.writerow([str(id),'[]'])
    else:
      tab.writerow([str(id),str([i for i, val in enumerate(val_results[id][0]) if val])])

f_out.close()